# INSTALLING DEPENDENCIES

In [ ]:
!pip install tensorflow pandas matplotlib sklearn

In [182]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [183]:
os.path.join('Toxicity-main','train.csv','train.csv')

'Toxicity-main\\train.csv\\train.csv'

In [184]:
df= pd.read_csv('PROJECT/train.csv')

In [185]:
df[df['toxic']==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [186]:
df.iloc[6]['comment_text']

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [187]:
df[df.columns[2:]].iloc[6]

toxic            1
severe_toxic     1
obscene          1
threat           0
insult           1
identity_hate    0
Name: 6, dtype: int64

# PREPROCESSING THE DATA

In [ ]:
!pip list

In [189]:
from tensorflow.keras.layers import TextVectorization

In [190]:
X= df['comment_text']
y= df[df.columns[2:]].values

In [191]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [192]:
df[df.columns[2:]].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [193]:
MAX_FEATURES =200000 

In [194]:
vectorizer = TextVectorization(max_tokens = MAX_FEATURES,
                              output_sequence_length = 1800,
                              output_mode = 'int')

In [ ]:
X.values

In [196]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer.get_vocabulary()

In [198]:
vectorized_text = vectorizer(X.values)

In [199]:
len(X)

159571

In [200]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [201]:
#MCSHBAP - map, cache, shuffle, batch,prefetch from tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [202]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [203]:
batch_y.shape

(16, 6)

In [204]:
int(len(dataset)*.7)

6981

In [205]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test= dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [206]:
len(train),len(val),len(test)

(6981, 1994, 997)

In [207]:
train_generator = train.as_numpy_iterator()

In [ ]:
train_generator.next()

# CREATING SEQUENTIAL MODEL

In [209]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [210]:
LSTM??

In [211]:
y[0]

array([0, 0, 0, 0, 0, 0], dtype=int64)

In [212]:
model = Sequential()
#creatin embeddin layer
model.add(Embedding(MAX_FEATURES+1, 32))
#Bidirectinal LSTM Layer
model.add(Bidirectional(LSTM(32, activation = 'tanh')))
#Feature extractor fully connected layers
model.add(Dense(128,  activation = 'relu'))
model.add(Dense(256,  activation = 'relu'))
model.add(Dense(128,  activation = 'relu'))
#Final layer
model.add(Dense(6,  activation = 'sigmoid'))


In [213]:
model.compile(loss= 'BinaryCrossentropy', optimizer = 'Adam')

In [214]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [ ]:
history = model.fit(train, epochs = 10, validation_data = val)

Epoch 1/10
6981/6981 [==============================] - 8918s 1s/step - loss: 0.0623 - val_loss: 0.0459
Epoch 2/10
 960/6981 [===>..........................] - ETA: 13:19:09 - loss: 0.0468

In [ ]:
history.history

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize = (16,4))
pd.DataFrame(history.history).plot()
plt.show()

# MAKING PREDICTIONS

In [ ]:
input_text = vectorizer('You freaking suck! I am going to kill you')

In [ ]:
input_text

In [ ]:
df.columns[2:]

In [ ]:
model.predict(np.array([input_text]))

In [ ]:
batch = test.as_numpy_iterator().next()

In [ ]:
batch_x,batch_y = test.as_numpy_iterator().next()

In [ ]:
batch_y

In [ ]:
model.predict(batch_x >0.5).astype(int)

In [ ]:
res = model.predict(np.expand_dims(input_text,0))

# EVALUATING MODEL

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator() :
    #upack the batch
    x_true, y_true = batch
    #makle a prediction
    yhat = model.predict(x_true)
    #flattenin the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true,yhat)
    re.update_state(y_true,yhat)
    acc.update_state(y_true,yhat)

In [ ]:
print(f'
      Precision: {pre.result().numpy()}, 
      Recall: {re.result().numpy()},
      Accuracy: {acc.result().numpy()}')

# TEST AND GRADIO

In [ ]:
!pip install gradio jinja2

In [ ]:
import gradio as gr
import tensorflow as tf

In [ ]:
model.save('toxicity.h5')

In [ ]:
model=tf.keras.models.load_model('toxicity.h5')

In [ ]:
input_str = vectorizer('Hey i freaking hate you! kill you and dump you')

In [ ]:
res= model.predict(np.expand_dims(input_str,0))

In [ ]:
df.columns[2:]

In [ ]:
res

In [ ]:
def score_comment(cmnt):
    vectorized_comment = vectorizer([cmnt])
    results = model.predict(vectorized_comment)
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += ' {}: {}\n'.format(col, results[0][idx]>0.5)
    return text

In [ ]:
interface = gr.Interface(fn = score_comment,
                        inputs=gr.inputs.Textbox(lines= 2, placeholder = 'comment to score'),
                         outputs='text')


In [ ]:
interface.launch(share=True)